In [3]:
import warnings
#Controle de avisos
warnings.filterwarnings("ignore")

# Installing CrewAI and CrewAI Tools

In [4]:
!python -m pip install crewai

In [5]:
!pip install crewai_tools

# Importing Necessary Libraries


In [1]:
from crewai import Agent, Task, Crew
import os
from IPython.display import Markdown
from google.colab import userdata


# Getting the API Key in a safe way
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# Setting the model to use for the CrewAI Agents
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

# Creating the Agents

In [2]:
# Creating the support agent using crewAI library
# Creates the agent using the Agent class, "role" defines its role, the "goal"
# sets the agent's objective and guides its actions, and the "backstory" is the
# backgroun and add context
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False, # Defines that the agent will perform the tasks by itself
	verbose=True # Defines that it will provide detailed log and messages
)

In [6]:
# Creating the support quality assurance agent using crewAI library
# Creates the agent using the Agent class, "role" defines its role, the "goal"
# sets the agent's objective and guides its actions, and the "backstory" is the
# backgroun and add context
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True # Defines that it will provide detailed log and messages
)

# Importing new tools from crewai_tools


In [7]:
# SerperDevTool is a tool likely used for searching the web or accessing specific APIs using the Serper search engine.
# ScrapeWebsiteTool is designed to extract information from websites by scraping their content.
# WebsiteSearchTool likely allows agents to search a specific website for information.
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

# Creating a local tool

In [8]:
# Define a local tool using the ScrapeWebsiteTool function and giving it the
# url from the site to be scraped
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

# Creating the first task

In [9]:
# Defining a task using the "task" class from crewai
# "description" provides the context and instructions for the task
# "expected_outputs" outlines the desired outcome
# "tools" defines the tools tath the agent can use for that task
# "agent" assigns the task to a agent
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

# Creating the second task

In [10]:
# Defining a task using the "task" class from crewai
# "description" provides the context and instructions for the task
# "expected_outputs" outlines the desired outcome
# "tools" defines the tools tath the agent can use for that task
# "agent" assigns the task to a agent
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

# Creating the Crew

In [16]:
# Creating the crew from the Crew class form crewai
# "agents" assing the agents to be used
# "tasks" defines the sequence of tasks to be executed in the correct order
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True, # Defines that it will provide detailed log and messages
  memory=True # make the agent use memory to increase the quality of its outcomes
)

/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Starting the Crew

In [17]:
# Defining an "inputs" dictionary, it will be used by the crew to perform its tasks
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
# begins the actions calling the "kickoff" function of the "crew" object providing
# the necessary context, the output is stored in the "result" variable
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 427d10cf-12d2-4323-87e0-725cf9952271                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 081791d2-e498-491d-a5ad-7665cebb142c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 081791d2-e498-491d-a5ad-7665cebb142c
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🚀 Crew: crew
└── 📋 Task: 081791d2-e498-491d-a5ad-7665cebb142c
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress
        └── 🧠 Thinking...

...

/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


🚀 Crew: crew
├── 📋 Task: 081791d2-e498-491d-a5ad-7665cebb142c
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Using Read website content (25)
└── 📋 Task: ab31fe2d-b9be-4028-be34-ddc49d637b48
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 081791d2-e498-491d-a5ad-7665cebb142c
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Using Read website content (25)
└── 📋 Task: ab31fe2d-b9be-4028-be34-ddc49d637b48
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ab31fe2d-b9be-4028-be34-ddc49d637b48                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 427d10cf-12d2-4323-87e0-725cf9952271                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Displaying the Result

In [21]:
# the markdown shows the clean output
Markdown(result.raw)

Hey there!

Thank you for reaching out to us with your inquiry about setting up a Crew in CrewAI and adding memory functionalities. We’re excited to help you get started! Here’s a detailed breakdown of the steps you need to follow to create and enhance your Crew:

1. **Create a Crew**: First, you’ll initialize a Crew using the CrewAI framework. This is done by calling specific functions in our API. You’ll need to define the composition of your Crew, including the roles each member will play and the tasks they’ll be responsible for. If you need guidance on API calls, feel free to check out our documentation [here](https://crewai.com/docs).

2. **Define Memory Capabilities**: To add memory functionalities, you will utilize the memory modules available in CrewAI. These modules allow your Crew to retain information from its past interactions and tasks. It’s essential to outline the structure of your memory—decide what type of data you want to store and how it will be retrieved. This way, your agents can reference previous conversations and learn from their experiences.

3. **Kick Off the Crew**: With your Crew and memory capabilities all set up, it’s time to kick off the Crew! This involves invoking the defined processes to activate the Crew, allowing agents to begin executing their assigned tasks. Ensure that all logistics are in place for a smooth start.

4. **Monitor and Adjust**: After launching your Crew, it’s crucial to monitor its performance and interactions actively. You can set up logging or use observability tools available in our platform that assist you in tracking how effectively memory is utilized and how well the agents are collaborating. This step is key to making any necessary adjustments for continuous improvement.

By following these steps, you’ll create a functional and adaptive Crew capable of learning from its past experiences! If you have any further questions or need specific examples, don’t hesitate to reach out. We’re here to help you every step of the way!

Best,
[Your Name]  
Support Team at CrewAI